<a href="https://colab.research.google.com/github/PedroMurta/Desafio-Dashboard/blob/main/Dashboard_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instanciar Banco de dados PostgreSQL

In [14]:
# Importar as bibliotecas que serão usadas durante o projeto
import psycopg2 # biblioteca para trabalhar com conexão do banco de dados
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go


In [11]:
# Configuração da conexão com o banco de dados no Railway
conn = psycopg2.connect(host='containers-us-west-59.railway.app',
                        dbname='railway', user='postgres', 
                        password='sMjvuuZSLqSDOYV84IcF',
                        port="6487")



# Leitura dos dados do desafio
position_data = pd.read_csv('fake_position.csv')
allocation_policies_data = pd.read_excel('fake_allocation_policies.xlsx')



# Criação das tabelas com as características dos arquivos lidos 
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE position (
            account_code FLOAT,
            account_suitability TEXT,
            asset_name TEXT,
            asset_cnpj TEXT,
            class_name TEXT,
            position_value FLOAT
            
             
        );
    """)
    cur.execute("""
        CREATE TABLE policy (            
            Classe TEXT,
            Conservador FLOAT,
            "Moderado Conservador" FLOAT,
            Moderado FLOAT,
            "Moderado Agressivo" FLOAT,
            Agressivo FLOAT,
            PRIMARY KEY (Classe)            
        );
    """)


# Inserção dos dados
with conn.cursor() as cur:
    for _, row in position_data.iterrows():
        cur.execute("""
            INSERT INTO position (account_code, account_suitability, asset_name, asset_cnpj, class_name, position_value)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['account_code'], row['account_suitability'], row['asset_name'], row['asset_cnpj'], row['class_name'],  row['position_value']))
    for _, row in allocation_policies_data.iterrows():
        cur.execute("""
            INSERT INTO policy (Classe, Conservador, "Moderado Conservador", Moderado, "Moderado Agressivo", Agressivo)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (row['Classe'], row['Conservador'], row['Moderado Conservador'], row['Moderado'], row['Moderado Agressivo'], row['Agressivo']))

# Salvar os dados e fazer o commit
conn.commit()


# Análise Exploratória dos Dados e Tratamento.

#### Entendendo a base de dados:

Os dados que serão usados neste projeto são dados fictícios do desafio proposto.

Os dados do arquivo *fake_position.csv* é composto por 6 colunas:

- `account_code`: código da conta do cliente

- `account_suitability`: perfil suitability da conta do cliente

- `asset_name`: nome do ativo financeiro na carteira do cliente
- `asset_cnpj`: CNPJ do ativo financeiro na carteira do cliente
- `class_name`: classe do ativo financeiro na carteira do cliente
- `position_value`: valor da posição do ativo financeiro na carteira do cliente

Os dados do arquivo *fake_allocation_policies.xlsx* também é composto por 6 colunas:

- `Classe`: nome da classe de ativos

- `Conservador`: percentual de alocação para a classe de ativos do perfil Conservador

- `Moderado Conservador`: percentual de alocação para a classe de ativos do perfil Moderado Conservador

- `Moderado`: percentual de alocação para a classe de ativos do perfil  Moderado

- `Moderado Agressivo`: percentual de alocação para a classe de ativos do perfil Moderado Agressivo

- `Agressivo`: percentual de alocação para a classe de ativos do perfil Agressivo

In [36]:
# Tabela Policies
df_policies = pd.read_excel('fake_allocation_policies.xlsx', names=['class_name', 'conservador', 'moderado-conservador', 'moderado', 'moderado-agressivo', 'agressivo'])

# Tabela Position
df_position = pd.read_csv('fake_position.csv')


# Tabela account 
accounts = df_position[['account_code', 'account_suitability']]

# Tabela asset 
asset = df_position['asset_name'].drop_duplicates()
asset = pd.DataFrame(asset)
asset.insert(0, 'id',  range(0, 0 + len(asset)))
asset.head()

account = accounts.drop_duplicates()

In [37]:
# Verificar os 5 primeiros valores da tabela position
df_position.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value
0,65547.00,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22
1,65547.00,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17
2,65547.00,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53
3,65547.00,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69
4,65547.00,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63


In [38]:
# Verificar os 5 primeiros valores da tabela policies
df_policies.head()

,class_name,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo
0,Renda Fixa Pós-Fixada,0.70,0.46,0.30,0.15,0.05
1,Renda Fixa Inflação,0.12,0.16,0.24,0.20,0.19
2,Renda Fixa Pré-Fixada,0.05,0.08,0.10,0.11,0.10
3,Renda Variável,0.02,0.04,0.07,0.14,0.20
4,Multimercado,0.09,0.21,0.22,0.30,0.31


In [39]:
# Verificar se possuímos valores nulos na tabela position
df_position.isnull().sum()

account_code             1
account_suitability     77
asset_name               1
asset_cnpj             348
class_name              11
position_value           1
dtype: int64

In [40]:
# Verificar se possuímos valores nulos na tabela policies
df_policies.isnull().sum()

class_name              0
conservador             0
moderado-conservador    0
moderado                0
moderado-agressivo      0
agressivo               0
dtype: int64

In [41]:
# Criar cópias de segurança caso aconteça algum imprevisto nas próximas etapas
position = df_position.copy()
policies = df_policies.copy()

# Como existe um valor que é nulo na coluna account_tipe, substituirei pelo valor 0  para facilitar a análise
position['account_code'].fillna(0, inplace=True)

# Transformar coluna account_code em int para facilitar a visualização 
position['account_code'] = position['account_code'].astype(int)

In [42]:
# Agrupar os dados por classe pare ter os valores de cada classe de ativos na carteira dos clientes
df_agrupado_classe = position.groupby(['account_code', 'account_suitability','class_name']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classe.head(9)

,account_code,account_suitability,class_name,position_value
0,2429,moderado,Renda Fixa Inflação,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,6425.97
3,2429,moderado,Reserva de Liquidez,13061.73
4,2429,moderado,Saldo em Conta,0.00
5,6777,moderado-conservador,Alternativos,32197.54
6,6777,moderado-conservador,Multimercado,20820.84
7,6777,moderado-conservador,Renda Fixa Inflação,32335.25
8,6777,moderado-conservador,Renda Fixa Pré-Fixada,18299.89


In [43]:
# Agrupar os dados pelo perfil suitability dos clientes para buscar os valores totais que cada cliente tem aplicado
df_agrupado_suitability = df_agrupado_classe.groupby(['account_code', 'account_suitability']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_suitability 

,account_code,account_suitability,position_value
0,2429,moderado,134626482.63
1,6777,moderado-conservador,470566.94
2,7826,moderado-conservador,183962.86
3,18073,moderado-conservador,65559838.14
4,20921,agressivo,27915.92
5,32263,moderado-conservador,278274189.45
6,32449,moderado-conservador,1726743.04
7,45848,moderado,125103.31
8,46168,moderado,162024.01
9,46744,moderado-agressivo,37913.63


In [45]:
df_agrupado_suitability['total_value'] = df_agrupado_suitability['position_value']

In [46]:
# Melhorar a visualização da relação do valor total de cada conta
total_agrupado = pd.merge(df_agrupado_suitability, df_agrupado_classe, on='account_code', how='left')
total_agrupado = total_agrupado[['account_code', 'total_value']].drop_duplicates()
total_agrupado.head()

,account_code,total_value
0,2429,134626482.63
5,6777,470566.94
13,7826,183962.86
19,18073,65559838.14
27,20921,27915.92


In [47]:
# Criar uma função que retorna a métrica de cada classe que busca os valores das colunas da tabela policies equivalentes a cada linha das colunas account_suitability + nome da classe

def get_metrica(row):
    politica = row['account_suitability']
    if politica == 'conservador':
        return row['conservador']

    elif politica == 'moderado conservador':
        return row['moderado_conservador']

    elif politica == 'moderado':
        return row['moderado']

    elif politica == 'moderado agressivo':
        return row['moderado_agressivo']

    elif politica == 'agressivo':
        return row['agressivo']
    else:
        return None

# Merge das duas tabelas 
merged = pd.merge(position, policies, on='class_name')

# Aplicar a função na tabela agrupada
merged['metrica'] = merged.apply(get_metrica, axis=1)

merged.head()

,account_code,account_suitability,asset_name,asset_cnpj,class_name,position_value,conservador,moderado-conservador,moderado,moderado-agressivo,agressivo,metrica
0,65547,moderado,CDB PRE DU CDB120CAG00,NaN,Renda Fixa Pré-Fixada,133886.22,0.05,0.08,0.10,0.11,0.10,0.10
1,65547,moderado,CDB PRE DU CDB320A2QHT,NaN,Renda Fixa Pré-Fixada,61127.17,0.05,0.08,0.10,0.11,0.10,0.10
2,65547,moderado,CDB PRE DU CDB420IBE9D,NaN,Renda Fixa Pré-Fixada,206855.53,0.05,0.08,0.10,0.11,0.10,0.10
3,65547,moderado,CDB PRE DU CDB321ALNBI,NaN,Renda Fixa Pré-Fixada,47910.69,0.05,0.08,0.10,0.11,0.10,0.10
4,65547,moderado,LCI PRE 21K00148900,NaN,Renda Fixa Pré-Fixada,71442.63,0.05,0.08,0.10,0.11,0.10,0.10


In [48]:
# Agrupar os dados pelas class_names para buscar a métrica e os valores atuais de cada ativo dos clientes
df_agrupado_classes = merged.groupby(['account_code', 'account_suitability','class_name', 'metrica']).agg({'position_value': 'sum'}).reset_index()
df_agrupado_classes.head(7)

,account_code,account_suitability,class_name,metrica,position_value
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97
3,2429,moderado,Saldo em Conta,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32
5,20921,agressivo,Renda Variável,0.20,4251.29
6,20921,agressivo,Saldo em Conta,0.00,3.65


In [49]:
# Criar nova tabela que insere os valores totais de acordo com cada account_code para facilitar cálculos futuros
df = pd.merge(df_agrupado_classes, total_agrupado, on='account_code')
df.head(7)

,account_code,account_suitability,class_name,metrica,position_value,total_value
0,2429,moderado,Renda Fixa Inflação,0.24,134310122.39,134626482.63
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,296872.54,134626482.63
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,6425.97,134626482.63
3,2429,moderado,Saldo em Conta,0.00,0.00,134626482.63
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,20237.32,27915.92
5,20921,agressivo,Renda Variável,0.20,4251.29,27915.92
6,20921,agressivo,Saldo em Conta,0.00,3.65,27915.92


In [50]:
# Renomear coluna position_value para valor_atual para facilitar a visualização
df['valor_atual'] = df['position_value']
df.drop(columns='position_value', inplace=True)
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual
0,2429,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97
3,2429,moderado,Saldo em Conta,0.00,134626482.63,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32
5,20921,agressivo,Renda Variável,0.20,27915.92,4251.29
6,20921,agressivo,Saldo em Conta,0.00,27915.92,3.65


In [51]:
# Para encontrar o valor esperado de cada classe dos clientes basta pegar o valor total e multiplicar pela métrica
df['valor_esperado'] = round(df['total_value'] * df['metrica'], 3)
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado
0,2429,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79
3,2429,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80
5,20921,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18
6,20921,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00


In [52]:
# Para facilitar a visualização novamente, criar uma coluna de valores atuais em porcentagem (%)
df['valor_porcentagem_atual'] = round(df['valor_atual'] / df['total_value'] * 100, 3)
df.head(13)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual
0,2429,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01
3,2429,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49
5,20921,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18,15.23
6,20921,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00,0.01
7,45848,moderado,Multimercado,0.22,125103.31,26015.53,27522.73,20.80
8,45848,moderado,Renda Fixa Inflação,0.24,125103.31,5604.31,30024.79,4.48
9,45848,moderado,Renda Fixa Pré-Fixada,0.10,125103.31,4161.47,12510.33,3.33


<img src='https://media.geeksforgeeks.org/wp-content/uploads/20210714205221/Photo1.jpg' width='500px' height='250px'>

In [53]:
# Encontrar o valor da distância para fazer o cálculo da Distância Euclidiana
# A fórmula da distância euclidiana está na imagem acima. elevar o resultado em 0.5 é a mesma coisa que tirar a raiz quadrada
df['distancia'] = ((df['valor_atual'] - df['valor_esperado']) ** 2)**0.5
df.head(7)

,account_code,account_suitability,class_name,metrica,total_value,valor_atual,valor_esperado,valor_porcentagem_atual,distancia
0,2429,moderado,Renda Fixa Inflação,0.24,134626482.63,134310122.39,32310355.83,99.77,101999766.56
1,2429,moderado,Renda Fixa Pré-Fixada,0.10,134626482.63,296872.54,13462648.26,0.22,13165775.72
2,2429,moderado,Renda Fixa Pós-Fixada,0.30,134626482.63,6425.97,40387944.79,0.01,40381518.82
3,2429,moderado,Saldo em Conta,0.00,134626482.63,0.00,0.00,0.00,0.00
4,20921,agressivo,Renda Fixa Pós-Fixada,0.05,27915.92,20237.32,1395.80,72.49,18841.52
5,20921,agressivo,Renda Variável,0.20,27915.92,4251.29,5583.18,15.23,1331.89
6,20921,agressivo,Saldo em Conta,0.00,27915.92,3.65,0.00,0.01,3.65


In [ ]:
"""import pandas as pd
import plotly.graph_objects as go

# Carregando os dados das posições e políticas
df_position = pd.read_csv('fake_position.csv')
df_policy = pd.read_excel('fake_allocation_policies.xlsx')

# Juntando os dados
df = pd.merge(df_position, df_policy, on='account_suitability')

# Calculando a diferença entre a alocação atual e desejada
df['diff'] = df['allocation'] - df['ideal_allocation']

# Agrupando por classe de ativos e calculando a média da diferença
df_grouped = df.groupby('asset_class')['diff'].mean().reset_index()

# Criando o gráfico de barras
fig = go.Figure(go.Bar(
    x=df_grouped['asset_class'],
    y=df_grouped['diff'],
    text=df_grouped['diff'],
    textposition='auto',
    marker_color='blue'
))

# Adicionando título e labels aos eixos
fig.update_layout(
    title='Diferença entre alocação atual e desejada por classe de ativos',
    xaxis_title='Classe de ativos',
    yaxis_title='Diferença média'
)

fig.show()
"""